In [ ]:
import pandas as pd
import os
import datetime as dt

# Marktstammdatenregisters

### Read data and combined

In [ ]:
def read_and_combine_files(start, end, folder_path):
    # Liste zum Speichern der eingelesenen DataFrames
    dfs = []

    # Durchlaufe den Bereich der Nummern
    for i in range(start, end + 1, 5000):
        # Erstelle den Dateinamen basierend auf dem Nummernbereich
        file_name = f"Stromerzeuger_{i}_bis_{i + 4999}.csv" 

        # Erstelle den vollständigen Pfad zur CSV-Datei
        file_path = os.path.join(folder_path, file_name)

        # Lese das CSV in einen DataFrame ein und füge ihn zur Liste hinzu
        df = pd.read_csv(file_path, delimiter=';')
        dfs.append(df)

    # Lese die letzte Datei ein
    last_file_path = os.path.join(folder_path, 'Stromerzeuger_465001_bis_465169.csv')
    last_df = pd.read_csv(last_file_path, delimiter=';')

    # Füge die letzte Datei zum DataFrame hinzu
    dfs.append(last_df)

    # Kombiniere alle DataFrames nach dem Index
    combined_df = pd.concat(dfs, axis=0, ignore_index=True)
    return combined_df

folder_path = r'C:\Users\mohdr\OneDrive\Desktop\BI specialist\Power BI\Portfolio Porject\Energyspeisung'

# Beispielaufruf der Funktion für den Bereich von 1 bis 189999
combined_df = read_and_combine_files(1, 464999, folder_path)

combined_df


# Applied mask, grouped by Inbetriebnahmedatum and sum of values

In [ ]:
mask_combined_df = (combined_df['Betriebs-Status'] == 'In Betrieb')  # Vergleichsoperator verwendet
df_combined_filtered = combined_df.loc[mask_combined_df, :]

# Display the filtered combined DataFrame

df_combined = pd.DataFrame({'date': pd.date_range(start='1/1/2015', end='12/31/2023', freq='1d')})


#df_combined = pd.DataFrame({'date': pd.date_range(start='1/1/2015', freq='1d', periods=365)})

df = pd.DataFrame(df_combined_filtered)  #um nur den gefilterten DataFrame zu verwenden

df_combined['Bruttoleistung'] = df['Bruttoleistung der Einheit'].str.replace(',', '.').astype(float)
df_combined['Nettonennleistung'] = df['Nettonennleistung der Einheit'].str.replace(',', '.').astype(float)

# Gruppiere nach Inbetriebnahmedatum und summiere die Werte
df_combined = df_combined.groupby('date').agg({'Bruttoleistung': 'sum','Nettonennleistung': 'sum'}).reset_index()

# Ergebnisse anzeigen
df_combined


# Bayern
### Tatsächliche und prognostizierte Solarenergieeinspeisung in der Regelzone von TenneT Deutschland - Bayern

In [ ]:
# data source: https://netztransparenz.tennet.eu/de/strommarkt/transparenz/transparenz-deutschland/netzkennzahlen/tatsaechliche-und-prognostizierte-solarenergieeinspeisung/bayern/
# filtered for 2023-01-01 to 2023-12-31

df_en_raw = pd.read_table('solarEnergyFeedIn_BY_2015-01-01_2023-12-31.csv',sep=';',parse_dates=['Datum'],decimal=',')

df_en = pd.DataFrame({'date': pd.date_range(start='1/1/2015', end='12/31/2023', freq='1d')})

df_en.loc[:,'Prog_in_MW'] = pd.DataFrame(df_en_raw.groupby(['Datum'], as_index=False)['Prognostiziert in MW'].sum()).iloc[:,1]
df_en.loc[:,'Act_in_MW'] = pd.DataFrame(df_en_raw.groupby(['Datum'], as_index=False)['Tatsaechlich in MW'].sum()).iloc[:,1]

#df = df.merge(df_en,on='date')
df_en

In [ ]:
df_en_com = pd.merge(df_combined,df_en, on = 'date')
df_en_com = pd.DataFrame(df_en_com)
df_en_com

# Stationsmessungen der Solarstrahlung

tageswerte_ST_03668 und tageswerte_ST_05792

In [ ]:
df = pd.DataFrame({'date': pd.date_range(start='1/1/2015', end= '1/1/2023', freq='1d')})

#'07370' Started 2021, '05404' Ended 2014 and '00867 Started 2021', therefore these stations are excluded

stationsid = ['03668','05792', '05856', '05705', '02290']
for id in stationsid:
    stationsname = 'stationid_'+id+'.txt'
    df_stat = pd.read_csv(stationsname,sep=';',parse_dates=['MESS_DATUM'],usecols=['MESS_DATUM','FD_STRAHL','FG_STRAHL','SD_STRAHL'])
    mask = (df_stat.loc[:,'MESS_DATUM'] >= '2015-01-01') & (df_stat.loc[:,'MESS_DATUM'] <= '2023-12-31')
    df_stat.columns =['date',id+'_FD_STRAHL',id+'_FG_STRAHL',id+'_SD_STRAHL']
    df = df.merge(df_stat.loc[mask,:],on='date')

df

# Final Combination DataFrame from 3 Tables

In [ ]:
df_final = df.merge(df_en_com, on = 'date')
df_final

In [ ]:
df_final.to_csv('df_final_2015_2023.csv')